In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Natural Language to Terminal Command Translator\n",
    "\n",
    "This notebook uses the Google Gemini API to convert plain English queries into executable terminal commands.\n",
    "\n",
    "### 🚨 **Security Notice** 🚨\n",
    "**Never** save your API key directly in the notebook. This code uses `getpass` to prompt you for your key securely, so it won't be visible or saved in the file's output."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 1: Install the required library\n",
    "!pip install -q google-generativeai"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Step 2: Import libraries and configure the API key\n",
    "import google.generativeai as genai\n",
    "import os\n",
    "from getpass import getpass\n",
    "\n",
    "# Securely get the API key from the user\n",
    "try:\n",
    "    # Check for an environment variable first\n",
    "    api_key = os.environ['GOOGLE_API_KEY']\n",
    "    print(\"Found API Key in environment variables.\")\n",
    "except KeyError:\n",
    "    # If not found, prompt the user securely\n",
    "    api_key = getpass('Enter your Google API Key: ')\n",
    "    os.environ['GOOGLE_API_KEY'] = api_key\n",
    "\n",
    "try:\n",
    "    genai.configure(api_key=api_key)\n",
    "    model = genai.GenerativeModel('gemini-1.5-flash-latest')\n",
    "    print(\"✅ Google AI initialized successfully!\")\n",
    "except Exception as e:\n",
    "    print(f\"❌ Error initializing Google AI: {e}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 3: Define the Translation Function\n",
    "\n",
    "This function takes your text query, builds a detailed prompt for the AI, sends the request, and cleans up the response to give you just the command."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def translate_to_command(natural_language_query):\n",
    "    \"\"\"Translates natural language into a shell command using the Gemini API.\"\"\"\n",
    "    \n",
    "    # List of safe, allowed commands for the AI to use\n",
    "    allowed_commands = [\n",
    "        'ls', 'cd', 'pwd', 'mkdir', 'rm', 'cat', 'grep', 'find', 'ps', 'top',\n",
    "        'tree', 'du', 'df', 'free', 'uptime', 'whoami', 'date', 'echo', 'touch',\n",
    "        'cp', 'mv', 'head', 'tail', 'sort', 'uniq', 'wc', 'curl', 'wget', 'ping'\n",
    "    ]\n",
    "\n",
    "    prompt = f\"\"\"\n",
    "    You are an expert system administrator. Your task is to convert a natural language request \n",
    "    into a single, executable bash command.\n",
    "    \n",
    "    - **Only** return the bash command.\n",
    "    - Do not include any explanation, preamble, or markdown formatting like ` ``` `.\n",
    "    - The command must be directly runnable.\n",
    "    - Only use these allowed commands: {', '.join(allowed_commands)}\n",
    "    - If the request is ambiguous or seems dangerous (like 'delete everything'), return \"Error: Ambiguous or unsafe request.\"\n",
    "\n",
    "    User Request: \"{natural_language_query}\"\n",
    "    \n",
    "    Command:\n",
    "    \"\"\"\n",
    "    \n",
    "    try:\n",
    "        response = model.generate_content(prompt)\n",
    "        # Clean up the response to remove backticks and whitespace\n",
    "        command = response.text.replace('`', '').strip()\n",
    "        return command\n",
    "    except Exception as e:\n",
    "        return f\"Error contacting AI model: {str(e)}\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 4: Test the Translator\n",
    "\n",
    "Now you can give it any query you want and see the resulting command."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- Example 1: Listing files ---\n",
    "query1 = \"show all files and folders in the current directory, including hidden ones\"\n",
    "command1 = translate_to_command(query1)\n",
    "\n",
    "print(f\"Your Query: '{query1}'\")\n",
    "print(f\"Generated Command: {command1}\\n\")\n",
    "\n",
    "# --- Example 2: Counting files ---\n",
    "query2 = \"count the number of python files here\"\n",
    "command2 = translate_to_command(query2)\n",
    "\n",
    "print(f\"Your Query: '{query2}'\")\n",
    "print(f\"Generated Command: {command2}\\n\")\n",
    "\n",
    "# --- Example 3: Finding text in a file ---\n",
    "query3 = \"find the line containing the word 'error' in the log file named app.log\"\n",
    "command3 = translate_to_command(query3)\n",
    "\n",
    "print(f\"Your Query: '{query3}'\")\n",
    "print(f\"Generated Command: {command3}\\n\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
